# BMG Labtech CLARIOstar Plus

| Summary | Photo |
|---------|-------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li><a href="https://www.bmglabtech.com/en/clariostar-plus/" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB</li> <li><b>Communication Level:</b> Firmware</li> <li><b>Measurement Modes:</b> Absorbance, Luminescence, Fluorescence</li> <li><b>Optical Systems:</b> Dual LVF Monochromator, Physical Filters, UV/Vis Spectrometer (220–1000 nm combined range)</li> <li><b>Plate Delivery:</b> Drawer</li> <li><b>Additional Features:</b> Temperature control, Shaking, Rapid full-plate autofocus, Configurable scan modes</li> <li>VID:PID <code>0403:BB68</code></li> </ul> | <div style="width:320px; text-align:center;"> ![clariostar](img/bmg-labtech-clariostar-plus.png) <br><i>Figure: BMG Labtech CLARIOstar Plus</i> </div> |

**CLARIOstar Plus vs CLARIOstar:** The CLARIOstar Plus (post-2019) replaced the original CLARIOstar. Both share the dual LVF Monochromator + filter + spectrometer architecture. The Plus adds rapid full-plate autofocus, newer PMT options (e.g. far-red), and Voyager control software. EDR (Enhanced Dynamic Range) was introduced after 2024 and is **not** present on all CLARIOstar Plus units. This backend targets the CLARIOstar Plus but may also work with the original CLARIOstar (untested).

---
## Setup Instructions (Physical)

The CLARIOstar and CLARIOstar Plus require a minimum of two cable connections to be operational:
1. Power cord (standard IEC C13)
2. USB cable (USB-B with security screws at CLARIOstar end; USB-A at control PC end)

Optional:
If you have a plate stacking unit to use with the CLARIOstar (Plus), an additional RS-232 port is available on the CLARIOstar (Plus).

---
## Setup Instructions (Programmatic)

To control the BMG Labtech CLARIOstar Plus, create a `PlateReader` frontend
instance that uses a `CLARIOstarPlusBackend` as its backend.

**Phase 1** supports: `setup()`, `open()`, `close()`, `stop()` and status polling.
Measurement commands (absorbance, fluorescence, luminescence) will be added in later phases.

In [ ]:
from pylabrobot import verbose

verbose(True)

In [ ]:
from pylabrobot.plate_reading import PlateReader
from pylabrobot.plate_reading.bmg_labtech import CLARIOstarPlusBackend

clariostar_plus_backend = CLARIOstarPlusBackend()

pr = PlateReader(
    name="CLARIOstar",
    backend=clariostar_plus_backend,
    size_x=0.0,
    size_y=0.0,
    size_z=0.0,
)

If you have multiple FTDI devices connected, pass a `device_id` to the backend to select the correct one:

```python
clariostar_plus_backend = CLARIOstarPlusBackend(device_id="FT1234AB")
```

In [ ]:
await pr.setup()

```{note}
Expected behaviour: the machine performs its initialization routine (FTDI connection, baudrate configuration,
initialize command). The backend polls status until the `initialized` flag is set.
```

---
## Defining a Plate

Every measurement requires a plate to be assigned to the plate reader.

In [ ]:
from pylabrobot.resources.corning import Cor_96_wellplate_360ul_Fb

plate = Cor_96_wellplate_360ul_Fb("test_plate")
pr.assign_child_resource(plate)

---
## Usage: Drawer

The CLARIOstar loads and unloads plates via a motorized drawer.

In [ ]:
# Open the drawer (plate out)
await pr.open()

In [ ]:
# Place your plate on the drawer (manually or via a robotic arm), then close it (plate in)
await pr.close()

---
## Future Tests

The cells below are commented out — they test features that are not yet implemented
in the Phase 1 backend. Uncomment and run them as each phase lands.

### Phase 2 — Device Identification

EEPROM config, firmware info, usage counters, status queries.

In [ ]:
# --- Phase 2: EEPROM machine configuration ---
config = clariostar_plus_backend.request_machine_config()
if config is not None:
    print(f"  {'model_name':25s} {config.model_name}")
    print(f"  {'serial_number':25s} {config.serial_number}")
    print(f"  {'firmware_version':25s} {config.firmware_version}")
    print(f"  {'firmware_build_timestamp':25s} {config.firmware_build_timestamp}")
    print(f"  {'machine_type_code':25s} 0x{config.machine_type_code:04x}")
    print(f"  {'has_absorbance':25s} {config.has_absorbance}")
    print(f"  {'has_fluorescence':25s} {config.has_fluorescence}")
    print(f"  {'has_luminescence':25s} {config.has_luminescence}")
    print(f"  {'has_alpha_technology':25s} {config.has_alpha_technology}")
    print(f"  {'monochromator_range':25s} {config.monochromator_range}")
    print(f"  {'num_filter_slots':25s} {config.num_filter_slots}")
else:
    print("No config available (EEPROM not read yet)")

In [ ]:
# --- Phase 2: Usage counters ---
counters = await clariostar_plus_backend.request_usage_counters()
for name, value in counters.items():
    print(f"  {name:25s} {value:>12,}")

In [ ]:
# --- Phase 2: Machine status ---
status = await clariostar_plus_backend.request_machine_status()
for flag, value in status.items():
    print(f"  {flag:20s} {value}")

In [ ]:
# --- Phase 2: Convenience status queries ---
if await clariostar_plus_backend.request_plate_detected():
    print("Plate is in the drawer")
else:
    print("No plate detected")

if await clariostar_plus_backend.request_busy():
    print("Machine is busy")
else:
    print("Machine is ready")

### Phase 3 — Temperature

Temperature monitoring, incubation control.

In [ ]:
# --- Phase 3: Read temperature ---
# temp = await clariostar_plus_backend.measure_temperature()
# print(f"Bottom plate: {temp:.1f} °C")
#
# temp_top = await clariostar_plus_backend.measure_temperature(sensor="top")
# print(f"Top plate:    {temp_top:.1f} °C")
#
# temp_mean = await clariostar_plus_backend.measure_temperature(sensor="mean")
# print(f"Mean:         {temp_mean:.1f} °C")

In [ ]:
# --- Phase 3: Incubation (heating to target) ---
# import asyncio
#
# await clariostar_plus_backend.start_temperature_control(37.0)
#
# for _ in range(5):
#     temp = await clariostar_plus_backend.measure_temperature()
#     print(f"Bottom plate: {temp:.1f} °C")
#     await asyncio.sleep(2)

### Phase 4 — Absorbance

Single-wavelength, multi-wavelength, transmittance, partial well selection.

In [ ]:
# --- Phase 4: Single-wavelength absorbance (OD) ---
# results = await pr.read_absorbance(
#     wavelength=600,
# )
#
# print(f"OD at 600nm, well A1: {results[0]['data'][0][0]}")

In [ ]:
# --- Phase 4: Absorbance as transmittance ---
# results = await pr.read_absorbance(
#     wavelength=450,
#     report="transmittance",
# )
#
# # Transmittance is in percent (0-100)
# print(f"Transmittance at 450nm, well A1: {results[0]['data'][0][0]}%")

In [ ]:
# --- Phase 4: Multi-wavelength absorbance ---
# results = await pr.read_absorbance(
#     wavelength=600,
#     wavelengths=[260, 280, 450, 600, 750],
# )
#
# for r in results:
#     print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]:.4f}")

In [ ]:
# --- Phase 4: Partial well selection (absorbance) ---
# column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]
#
# results = await pr.read_absorbance(
#     wavelength=600,
#     wells=column_1_wells,
# )

### Phase 5 — Fluorescence

Basic, custom gain/bandwidth, bottom-optic.

In [ ]:
# --- Phase 5: Basic fluorescence (GFP: ex 485 / em 528) ---
# results = await pr.read_fluorescence(
#     excitation_wavelength=485,
#     emission_wavelength=528,
#     focal_height=8.5,
# )
#
# print(f"GFP fluorescence, well A1: {results[0]['data'][0][0]}")

In [ ]:
# --- Phase 5: Fluorescence with custom gain, bandwidth, flashes ---
# results = await pr.read_fluorescence(
#     excitation_wavelength=485,
#     emission_wavelength=528,
#     focal_height=8.5,
#     gain=1500,
#     ex_bandwidth=10,
#     em_bandwidth=20,
#     flashes=50,
# )

In [ ]:
# --- Phase 5: Bottom-optic fluorescence ---
# results = await pr.read_fluorescence(
#     excitation_wavelength=544,
#     emission_wavelength=590,
#     focal_height=4.5,
#     bottom_optic=True,
# )

### Phase 6 — Luminescence

Basic luminescence, partial well selection.

In [ ]:
# --- Phase 6: Basic luminescence (full plate) ---
# results = await pr.read_luminescence(
#     focal_height=13.0,
# )
#
# print(f"Temperature: {results[0]['temperature']:.1f} °C")
# print(f"Well A1 RLU: {results[0]['data'][0][0]}")

In [ ]:
# --- Phase 6: Partial well luminescence ---
# selected_wells = [plate.get_well("A1"), plate.get_well("D6"), plate.get_well("H12")]
#
# results = await pr.read_luminescence(
#     focal_height=13.0,
#     wells=selected_wells,
# )

### Phase 7 — Advanced Features

Scan modes, shaking, non-blocking reads, combined features.

In [ ]:
# --- Phase 7: Scan mode configuration ---
# from pylabrobot.plate_reading.bmg_labtech.clariostar_plus_backend import StartCorner
#
# results = await pr.read_absorbance(
#     wavelength=600,
#     start_corner=StartCorner.BOTTOM_RIGHT,
#     unidirectional=True,
#     vertical=True,
# )

In [ ]:
# --- Phase 7: Pre-measurement shaking ---
# from pylabrobot.plate_reading.bmg_labtech.clariostar_plus_backend import ShakerType
#
# results = await pr.read_luminescence(
#     focal_height=13.0,
#     shake_type=ShakerType.ORBITAL,
#     shake_speed_rpm=300,
#     shake_duration_s=5,
# )

In [ ]:
# --- Phase 7: Non-blocking read ---
# import asyncio
#
# await pr.read_absorbance(
#     wavelength=600,
#     wavelengths=[260, 280, 450, 600],
#     wait=False,
# )
#
# status = await clariostar_plus_backend.request_machine_status()
# while not status["unread_data"]:
#     print("Measuring...")
#     await asyncio.sleep(1)
#     status = await clariostar_plus_backend.request_machine_status()
#
# results = await clariostar_plus_backend.collect_absorbance_measurement(plate)

In [ ]:
# --- Phase 7: Combined features ---
# row_A = [plate.get_well(f"A{col}") for col in range(1, 13)]
#
# results = await pr.read_fluorescence(
#     excitation_wavelength=544,
#     emission_wavelength=590,
#     focal_height=6.0,
#     wells=row_A,
#     gain=2000,
#     shake_type=ShakerType.ORBITAL,
#     shake_speed_rpm=300,
#     shake_duration_s=3,
#     start_corner=StartCorner.BOTTOM_RIGHT,
# )

---
## Reference: How Absorbance Calculation Works

The machine does not measure absorbance directly. It measures **raw detector counts**
and the backend converts them to transmittance and then to OD.
See `DESIGN.md` in `pylabrobot/plate_reading/bmg_labtech/` for the full protocol reference.

---
## Closing Connection

In [ ]:
pr.unassign_child_resource(plate)

In [ ]:
await pr.stop()

This closes the FTDI connection. After calling `stop()`, you must call `setup()` again before using the plate reader.